# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
import pickle

In [73]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [4]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [6]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [ ]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[4733:6999]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

In [66]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,isbn,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,9791160576658,[9.6],"[85%, 15%, 0%, 0%, 0%]","[일일호시일의 원작자라...., 위로를 주는 책, 읽고 친구 선물했어요, 계절에 따...","[민**빠, e*******4, y*******8, 모*, l******e, l**...","[5점, 5점, 5점, 4점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n[일일시호일]의 원작자로 국내 독자에게도 눈도장을 찍은 인기 에세이스트 모리시...
1,9788934981633,[8.7],"[57%, 29%, 14%, 0%, 0%]","[[책] 너무 늦기 전에 들어야 할 임종학 강의 _ 죽음을 받아들이는 법, 임종학 ...","[g*****9, 로*리, t******7, s*****5, R**o, 책*****...","[5점, 5점, 5점, 4점, 4점, 4점, 3점]",[\n \n\n \n \n5월은 가정의 달이다. 우리는 \n저마다 가정을 일구고 있...
2,9788960780484,[9.0],"[100%, 0%, 0%, 0%, 0%]","[삶의 향기 몇 점 - 황동규, 향기로운 산문]","[l******2, 책****벤]","[5점, 5점]","[\n황동규...\n \n시인. 내가 누군가에게 늘 황동규 시인을 소개를 할 때면,..."
3,9791196866907,[9.7],"[91%, 9%, 0%, 0%, 0%]","[제 주치의 선생님이 쓰신 책, 마음이 따뜻해지는 책, 리뷰, 너무나도 자신과 타인...","[헤*즐, k*****1, d***a, 뽀*****송, n***a, 어*맘, Y**...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n5년 넘게 진료받고 있다가 제가 존경하고 항상 도움받고 있던 선생님이 내신 책...
4,9788946420861,[8.9],"[56%, 38%, 3%, 3%, 0%]","[[서평]곱게 늙기-송차선, [서평] 곱게 늙기 [샘터], 곱게 늙기, OLYMPI...","[나*, 세*, 공*, k*******7, y******0, 북*더, y*b, 가*...","[5점, 5점, 5점, 4점, 4점, 4점, 4점, 4점, 5점, 5점]",[\n사람들은 누구나 아이로 태어나서 나이 들어 죽음을 맞이하게 된다. 그것이 가장...
5,9791157841516,[9.3],"[78%, 22%, 0%, 0%, 0%]","[정말 대단하십니다, 어르신들..., 보고싶은 할머니, 어머니..... 너무 순수해...","[눈*, 푸*미, w*****e, 청****1, 바**사, 꺌*루, 소*, q***...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]","[\n책을 읽은 지는 꽤 됐습니다만, 느낌을 정리하는 것이 쉽지 않아 리뷰를 미루고..."
6,9788965745075,[9.6],"[84%, 16%, 0%, 0%, 0%]","[필요한 한마디 ""자뻑은 나의 힘"", 자뻑은 나의 힘, 자뻑은 나의 힘, 자뻑은 나...","[유*맘, k***i, 코***쥬, 별**이, 두**리, k***s, 오*, b**...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n긴머리를 질끈 묶은 모습이 인상적인 기인으로만 느껴지던 작가님이 언제부터인가 ...
7,9788925589541,[9.6],"[80%, 20%, 0%, 0%, 0%]","[희망으로 꽃을 피워, 희망으로 꽃을 피워, 희망으로 꽃을 피워, 엄마로서 행복하고...","[m**********m, s******i, c*******3, 엔***어, l**...","[5점, 5점, 5점, 5점, 4점]","[\n\n \n\n \n책이 주는 느낌부터 다채롭다. 화려한 그림이 눈에 띄지만, ..."
8,9788952779595,[9.1],"[70%, 30%, 0%, 0%, 0%]","[[서평] 서로의 마음을 산책 중 _ 자토, [서로의 마음을 산책중]달달하고 귀여운...","[꽃*럼, 알*, w*******7, w********0, 예*리, 다*냥, j**...","[5점, 5점, 5점, 5점, 4점, 5점, 4점, 4점, 4점, 4점]",[\n 요즘 책이 증~~~~말루 안 읽힌다. 시험 본다고 근 몇 주 동안...
9,9788956607047,[7.8],"[20%, 60%, 20%, 0%, 0%]","[나의 핀란드 여행 : 카모메 식당 뒷이야기, 즐거움이 듬뿍~, 나의 핀란드 여행,...","[토*, 사**미, 노*타, d****7, 가*, h******3, 에**스, 보*...","[4점, 4점, 4점, 5점, 4점, 5점, 4점, 4점, 4점, 4점]",[\n이 책의 번역본이 출간된 건 2013년 7월이다. 어렴풋 하지만 분명 당시에 ...


In [67]:
for i in range(4681,6999):
    if isbn13[i] == '9791197080876':
        print(i)

4731


In [68]:
user.to_pickle('user_4732-4731.pkl', protocol = 4)

In [15]:
user1 = pd.read_pickle('user_3041-3383.pkl')
user2 = pd.read_pickle('user_3384-3427.pkl')